# Company webpage

Extract some key information from the company webpage

In [1]:
from core import CompanyProduct, init

init()

In [2]:
target = CompanyProduct.same("98point6")

In [3]:
from core import make_experiment_dir

experiment_dir = make_experiment_dir(target)

In [17]:
from google_search import search

results = list(search("site:98point6.com about", num=1))
results

2024-08-19 14:58:11.092 | DEBUG    | search:search:65 - Google search results: {'kind': 'customsearch#search', 'url': {'type': 'application/json', 'template': 'https://www.googleapis.com/customsearch/v1?q={searchTerms}&num={count?}&start={startIndex?}&lr={language?}&safe={safe?}&cx={cx?}&sort={sort?}&filter={filter?}&gl={gl?}&cr={cr?}&googlehost={googleHost?}&c2coff={disableCnTwTranslation?}&hq={hq?}&hl={hl?}&siteSearch={siteSearch?}&siteSearchFilter={siteSearchFilter?}&exactTerms={exactTerms?}&excludeTerms={excludeTerms?}&linkSite={linkSite?}&orTerms={orTerms?}&dateRestrict={dateRestrict?}&lowRange={lowRange?}&highRange={highRange?}&searchType={searchType}&fileType={fileType?}&rights={rights?}&imgSize={imgSize?}&imgType={imgType?}&imgColorType={imgColorType?}&imgDominantColor={imgDominantColor?}&alt=json'}, 'queries': {'request': [{'title': 'Google Custom Search - site:98point6.com about', 'totalResults': '39', 'searchTerms': 'site:98point6.com about', 'count': 1, 'startIndex': 1, 'la

[SearchResult(title='About Us | 98point6 Technologies', link='https://www.98point6.com/about-us/', snippet='A software company powering best in class virtual care experiences with purpose-built technology advancing medical technology to enable exceptional\xa0...', formattedUrl='https://www.98point6.com/about-us/')]

In [18]:
# request and get the content
from news.scrape import request_article, response_to_article

response = request_article(results[0].link)
content = response_to_article(response)
content

In [19]:
from news.scrape import article_to_markdown

print(article_to_markdown(content))

# [98point6 Technologies on 2023-12-19](https://www.98point6.com/about-us/)
Yes, we used to be affiliated with a virtual clinic â¦

Before the days when video calls and remote meetings were ubiquitous, our team of healthcare experts knew there was a need to expand access to quality care beyond physical spaces. In 2017, 98point6 launched technology that created a 24/7 virtual clinic with on-demand, text-based primary care. Over the years, we refined the platform to:

Integrate with electronic health records

Provide clinical documentation and decision-support tools

Improve provider efficiency

â¦ and now our software is changing healthcare tech.

98point6 is no longer affiliated with a virtual clinic, and in 2023, we evolved to focus our business on building and licensing technology to healthcare organizations. In short, weâve made it even easier for customers to leverage our learnings and experience for their virtual care needs.

The fine print

In spring 2023, Transcarent acquire